# Naive Bayes and Sentiment Classification

Classifier for text classification

- Input: $d$ ("document")
- Output: $c$ ("class")
- Training set: $N$ documents that have each been hand-labeled with a class $(d_1, c_1), \dots, (d_N, c_N)$

- 🎯 Goal: to learn a classifier that is capable of mapping from a new document $d$ to its correct class $c\in C$

- Type:
  - Generative: 
    - Build a model of how a class could generate some input data
    - Given an observation, return the class most likely to have generated the observation.
    - E.g.: Naive Bayes
  - Discriminative
    - Learn what features from the input are most useful to discriminate between the different possible classes
    - Often more accurate and hence more commonly used
    - E.g.: Logistic Regression



## Naive Bayes Classifier

We represent a text document as if were a **bag-of-words**

- unordered set of words
- position ignored
- keeping only their frequency in the document

<img src="https://raw.githubusercontent.com/EckoTan0804/upic-repo/master/uPic/截屏2020-06-14%2011.55.44.png" alt="截屏2020-06-14 11.55.44" style="zoom:80%;" />

> In the example in the figure, instead of representing the word order in all the phrases like “I love this movie” and “I would recommend it”, we simply note that the word *I* occurred 5 times in the entire excerpt, the word *it* 6 times, the words *love*, *recommend*, and *movie* once, and so on.

Naive Bayes:

- Probablistic classifier 

  - for a document $d$, out of all classes $c \in C$, the classifier returns the class $\hat{c}$ which has the **maximum a-posterior probability (MAP)** given the document

  $$
  \begin{array}{ll}
  \hat{c} &= \underset{c \in C}{\operatorname{argmax}} P(c | d) \\
  &\overset{\text{Bayesian inference}}{=} \underset{c \in C}{\operatorname{argmax}} \frac{P(d | c) P(c)}{P(d)} \\
  &= \underset{c \in C}{\operatorname{argmax}} \underbrace{P(d | c)}_{\text{likelihood}} \underbrace{P(c)}_{\text{prior}} \\
  
  \end{array}
  $$

We represent a document $d$ as a set of features $f_1, f_2, \dots, f_n$, then
$$
\hat{c}=\underset{c \in C}{\operatorname{argmax}} \overbrace{P\left(f_{1}, f_{2}, \ldots, f_{n} | c\right)}^{\text {likelihood }} \overbrace{P(c)}^{\text {prior }}
$$
In order to make it possible to compute directly, Naive Bayes classifiers make two simplifying assumptions

- Bag-of-words assumption
  -  we assume position does NOT matter, and that the word “love” has the SAME effect on classification whether it occurs as the 1st, 20th, or last word in the document. 
  - Thus we assume that the features $f_1, f_2, \dots, f_n$ only encode word identity and not position.

- Naive Bayes assumption

  - The conditional independence assumption that the probabilities $P(f_i|c)$ are independent given the class $c$ and hence can be ‘naively’ multiplied
    $$
    P\left(f_{1}, f_{2}, \ldots ., f_{n} | c\right)=P\left(f_{1} | c\right) \cdot P\left(f_{2} | c\right) \cdot \ldots \cdot P\left(f_{n} | c\right)
    $$

Based on these two assumptions, the final equation for the class chosen by a naive Bayes classifier is thus
$$
c_{N B}=\underset{c \in C}{\operatorname{argmax}} P(c) \prod_{f \in F} P(f | c)
$$
To apply the naive Bayes classifier to text, we need to consider word positions, by simply walking an index through every word position in the document:
$$
\text{positions} \leftarrow \text{all word positions in test document}\\
c_{NB}=\underset{c \in C}{\operatorname{argmax}} P(c) \prod_{i \in \text {positions}} P\left(w_{i} | c\right)
$$
To avoid underflow and increase speed, Naive Bayes calculations, like calculations for language modeling, are done in log space:
$$
c_{NB}=\underset{c \in C}{\operatorname{argmax}} \log P(c)+\sum_{i \in  \text {postiions }} \log P\left(w_{i} | c\right)
\label{eq:NB log}
$$


## Training the Naive Bayes Classifier

In $\eqref{eq:NB log}$ we have to learn the probabilities $P(c)$ and $P(w_i|c)$. We use the **Maximum Likelihood Estimate (MLE)** to estimate them. We’ll simply use the **frequencies** in the data.

- $P(c)$: document prior

  - "what percentage of the documents in our training set are in each class $c$ ?"

  $$
  \hat{P}(c)=\frac{N_{c}}{N_{d o c}}
  $$

  - $N_c$: the number of documents in our training data with class $c$
  - $N_{doc}$: the total number of documents

- $P(w_i|c)$

  - "The fraction of times the word $w_i$ appears among all words in all documents of topic $c$"

    - We first concatenate all documents with category $c$ into one big “category $c$” text.

    - Then we use the frequency of $w_i$ in this concatenated document to give a maximum likelihood estimate of the probability
      $$
      \hat{P}\left(w_{i} | c\right)=\frac{\operatorname{count}\left(w_{i}, c\right)}{\sum_{w \in V} \operatorname{count}(w, c)}
      $$

      - $V$: vocabulary that consists of the union of all the word types in all classes, not just the words in one class $c$

  - Avoid zero probablities in the likelihood term for any class: **Laplace (add-one) smoothing**
    $$
    \hat{P}\left(w_{i} | c\right)=\frac{\operatorname{count}\left(w_{i}, c\right)+1}{\sum_{w \in V}(\operatorname{count}(w, c)+1)}=\frac{\operatorname{count}\left(w_{i}, c\right)+1}{\left(\sum_{w \in V} \operatorname{count}(w, c)\right)+|V|}
    $$

    > Why is this a problem?
    >
    > Imagine we are trying to estimate the likelihood of the word “fantastic” given class *positive*, but suppose there are no training documents that both contain the word “fantastic” and are classified as *positive*. Perhaps the word “fantastic” happens to occur (sarcastically?) in the class *negative*. In such a case the probability for this feature will be zero:
    > $$
    > \hat{P}(\text { "fantastic" } | \text { positive })=\frac{\text { count }(\text { "fantastic } " \text { , positive })}{\sum_{w \in V} \text { count }(w, \text { positive })}=0
    > $$
    > But since naive Bayes naively multiplies all the feature likelihoods together, zero probabilities in the likelihood term for any class will cause the probability of the class to be zero, no matter the other evidence!

In addition to $P(c)$ and $P(w_i|c)$, We should also deal with

- **Unknown words** 
  - Words that occur in our test data but are NOT in our vocabulary at all because they did not occur in any training document in any class
  - 🔧 Solution: Ignore them
    - Remove them from the test document and not include any probability for them at all
- **Stop words**
  - Very frequent words like *the* and *a*
  - Solution:
    - Method 1: 
      1. sorting the vocabulary by frequency in the training set
      2. defining the top 10–100 vocabulary entries as stop words
    - Method 2: 
      1. use one of the many pre-defined stop word list available online
      2. Then every instance of these stop words are simply removed from both training and test documents as if they had never occurred.
  - In most text classification applications, however, using a stop word list does NOT improve performance 🤪, and so it is more common to make use of the entire vocabulary and not use a stop word list.

### Example

We’ll use a sentiment analysis domain with the two classes positive (+) and negative (-), and take the following miniature training and test documents simplified from actual movie reviews.

![截屏2020-06-14 12.37.43](https://raw.githubusercontent.com/EckoTan0804/upic-repo/master/uPic/截屏2020-06-14%2012.37.43.png)

- The prior $P(c)$:
  $$
  P(-)=\frac{3}{5} \quad P(+)=\frac{2}{5}
  $$

The word *with* doesn’t occur in the training set, so we drop it completely.

The remaining three words are "predictable", "no", and "fun". Their likelihoods from the training set are (with Laplace smoothing): 
$$
\begin{aligned}
P\left(\text { "predictable" }|-\right) &=\frac{1+1}{14+20} \quad P\left(\text { "predictable" } |+\right)=\frac{0+1}{9+20} \\
P\left(\text{"no"}|-\right) &=\frac{1+1}{14+20} \quad P\left(\text{"no"}|+\right)=\frac{0+1}{9+20} \\
P\left(\text{"fun"}|-\right) &=\frac{0+1}{14+20} \quad P\left(\text{"fun"}|+\right)=\frac{1+1}{9+20}
\end{aligned}
$$

> $$
> V=\{\text{just, plain, boring, entirely, predictable, and, lacks, energy, no, suprises, very, few, laughs, powerful, the, most, fun, film, of, summer}\}
> $$
>
> $\Rightarrow |V|=20$
>
> The word "predictable" occurs in negative (-) once, therefore, with Laplace smoothing:
>
> $P\left(\text { "predictable" }|-\right) =\frac{1+1}{14+20}$

For the test sentence $S=$ "predictable with no fun", after removing the word "with":
$$
\begin{array}{l}
P(-) P(S |-)=\frac{3}{5} \times \frac{2 \times 2 \times 1}{34^{3}}=6.1 \times 10^{-5} \\
P(+) P(S |+)=\frac{2}{5} \times \frac{1 \times 1 \times 2}{29^{3}}=3.2 \times 10^{-5}
\end{array}
$$
$P(-)P(S|-) > P(+)P(S|+)$ 

$\Rightarrow$ The model predicts the class *negative* for the test sentence $S$.



## Optimizing for Sentiment Analysis

While standard naive Bayes text classification can work well for sentiment analysis, some small changes are generally employed that improve performance. 💪

### Binary multinomial naive Bayes (binary NB)

First, for sentiment classification and a number of other text classification tasks, whether a word occurs or not seems to matter more than its frequency. 

**Thus it often improves performance to clip the word counts in each document at 1**.

Example:

- The example is worked without add-1 smoothing to make the differences clearer. 
- Note that the results counts need not be 1; the word *great* has a count of 2 even for Binary NB, because it appears in multiple documents (in both positive and negative class).

<img src="https://raw.githubusercontent.com/EckoTan0804/upic-repo/master/uPic/截屏2020-06-14%2012.55.28.png" alt="截屏2020-06-14 12.55.28" style="zoom:80%;" />

### Deal with negation

During text normalization, prepend the prefix *NOT_* to every word after a token of logical negation (*n’t, not, no, never*) until the next punc- tuation mark. 

Example:

![截屏2020-06-14 12.58.03](/Users/EckoTan/Library/Application Support/typora-user-images/截屏2020-06-14 12.58.03.png)

### Deal with insufficient labeled training data

Derive the positive and negative word features from sentiment lexicons, lists of words that are pre-annotated with positive or negative sentiment.

Popular lexicons:

- General Inquirer
- LIWC
- opinion lexicon
- MPQA Subjectivity Lexicon



## Evaluation

**Gold labels**: the human-defined labels for each document that we are trying to match

To evaluate any system for detecting things, we start by building a **Contingency table (Confusion matrix)**:

<img src="https://raw.githubusercontent.com/EckoTan0804/upic-repo/master/uPic/截屏2020-06-14%2013.42.20.png" alt="截屏2020-06-14 13.42.20" style="zoom:80%;" />

Evaluation metric:

- **Accuracy**
  $$
  \text{Accuracy} = \frac{\text{tp+tn}}{\text{tp+fp+tn+fn}}
  $$

  - doesn’t work well when the classes are unbalanced 🤪

- **Precision (P)**

  - measures the percentage of the items that the system detected (i.e., the system labeled as positive) that are in fact positive (i.e., are positive according to the human gold labels).
    $$
    \text{Precision} = \frac{\text{tp}}{\text{tp+fp}}
    $$

- **Recall (R)**

  - measures the percentage of items actually present in the input (i.e., are positive according to the human gold labels) that were correctly identified by the system (i.e., the system labeled as positive).
    $$
    \text{Recall} = \frac{\text{tp}}{\text{tp+fn}}
    $$

- **F-measure**
  $$
  F_{\beta}=\frac{\left(\beta^{2}+1\right) P R}{\beta^{2} P+R}
  $$

  - $\beta$: differentially weights the importance of recall and precision, based perhaps on the needs of an application

    - $\beta > 1$: favor recall

    - $\beta < 1$: favor precision

    - $\beta = 1$: precision and recall are equally balanced (the most frequently used metric)
      $$
      F_{1}=\frac{2 P R}{P+R}
      $$

### More than two classes

Solution: **one-of** or **multinomial classification**

- The classes are **mutually exclusive** and each document or item appears in exactly ONE class.
- How it works?
  - We build a separate binary classifier trained on positive examples from $c$ and negative examples from all other classes. 
  - Given a test document or item $d$, we run all the classifiers and choose the label from the classifier with the highest score.

E.g.:

<img src="https://raw.githubusercontent.com/EckoTan0804/upic-repo/master/uPic/截屏2020-06-14%2013.58.26.png" alt="截屏2020-06-14 13.58.26" style="zoom:80%;" />

Evaluation metric:

- **Macroaveraging**
  1. compute the performance for each class
  2. then average over classes
- **Microaveraging**
  1. collect the decisions for all classes into a single contingency table
  2. then compute precision and recall from that table.

E.g.: 

![截屏2020-06-14 14.00.23](https://raw.githubusercontent.com/EckoTan0804/upic-repo/master/uPic/截屏2020-06-14%2014.00.23.png)

